In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import agentpy as ap
import random
import IPython
from math import sqrt,exp

#serve se si vuole fare animazioni più grandi, i.e. più step. Si assegna il valore massimo che può occupare in memoria, in Mb
mpl.rcParams['animation.embed_limit'] = 500

class Cell(ap.Agent):

    """ Higher level class """

    def update_position(self):
        self.neighbors = self.space.neighbors(self)
        self.pos = self.space.positions[self]

    def oxygen_update(self):
        self.oxy = self.space.oxy[self.pos]

    def aging(self):
        self.phase += self.p.time_step * (self.oxy/(self.p.Tmin * (self.p.C_phase + self.oxy)))

    def move(self):
        self.neighbors = self.space.neighbors(self)
        empty = self.empty_cells(self.pos)
        if empty and random.random() < self.move_prob:
            new_pos = random.choice(empty)
            self.space.move_to(self,pos = new_pos) 

    def empty_cells(self,pos,distance = 1):
        """
        Empty cells finds all available grid cells in a neighborhood around the agent
        whose radius (distance) is given as a parameter, and incluedes diagonal cells also
        """
        x,y = pos  
        rng = tuple(range(-distance,distance+1))
        moore = [(x+i, y+j) for i in rng for j in rng if (i, j) != (0, 0)]
        neighbors = [p for p in moore if all(0 <= coord < self.p.size for coord in p)]
        empty_cells = [tuple for tuple in neighbors if tuple in self.space.empty]
        return empty_cells     

    def death(self):
        self.space.remove_agents(self)
        self.model.agents.remove(self)

class CancerCell(Cell):
    
    def go_sleep(self):
        
        pos = self.pos
        self.space.remove_agents(self)
        self.model.agents.remove(self)
        sleepy_cancer = ap.AgentList(self.model,1,QuiescentCancerCell)
        sleepy_cancer.PDL1 = self.PDL1
        sleepy_cancer.phase = self.phase
        self.space.add_agents(sleepy_cancer,positions = [pos])
        sleepy_cancer.update_position()  
        self.model.agents.extend(sleepy_cancer)

    def death(self):
        self.space.remove_agents(self)
        self.model.agents.remove(self)
        self.model.dead_cancer[-1] += 1

class QuiescentCancerCell(CancerCell):
    
    def setup(self): 
        self.idc = 0 
        self.space = self.model.grid
        self.age = 0 
        self.move_prob = 0
        self.PDL1 = False
        
    def wake_up(self):
        pos = self.pos
        self.space.remove_agents(self)
        self.model.agents.remove(self)
        awake_cancer = ap.AgentList(self.model,1,ProliferativeCancerCell)
        awake_cancer.PDL1 = self.PDL1
        awake_cancer.phase = self.phase
        self.space.add_agents(awake_cancer,positions = [pos])
        awake_cancer.update_position() 
        self.model.agents.extend(awake_cancer)        

class ProliferativeCancerCell(CancerCell):

    def setup(self):
        self.idc = 1
        self.phase = 0
        self.space = self.model.grid
        self.move_prob = self.p.move_prob_Cancer
        self.PDL1 = False
        
    def replication(self):

        self.neighbors = self.space.neighbors(self) #update neighbors bc neighbors might have been occupied by previous replications
        empty_cells = self.empty_cells(self.pos)
        if empty_cells:
            new_pos = [random.choice(empty_cells)]
            new_cancer = ap.AgentList(self.model,1,ProliferativeCancerCell)
            new_cancer[0].PDL1 = self.PDL1 #mutations are inherited
            self.space.add_agents(new_cancer,positions = new_pos)
            self.update_position()   
            new_cancer.update_position()
            self.phase = 0  
            self.model.agents.extend(new_cancer)

class TCell(Cell):             
    
    def replication(self):
        self.neighbors = self.space.neighbors(self)  #update neighbors bc neighbors might have been occupied by previous replications
        empty_cells = self.empty_cells(self.pos)
        self.IL2 = self.space.il2[self.pos]
        if empty_cells  and self.IL2 > self.p.il2_threshold:
            new_pos = [random.choice(empty_cells)]
            new_tcell = ap.AgentList(self.model,1,self.__class__)
            self.space.add_agents(new_tcell,positions = new_pos)
            self.update_position()   
            new_tcell.update_position()            
            self.phase = 0
            self.model.agents.extend(new_tcell)

class EffectorTCell(TCell):

    def setup(self):
        self.idc = 3
        self.phase = 0
        self.space = self.model.grid
        self.move_prob = self.p.move_prob_TCell
        
        

    def activation(self):
        detected_cancer = self.model.selection(self.neighbors,ProliferativeCancerCell)
        if detected_cancer:
            pos = self.pos
            self.space.remove_agents(self)
            self.model.agents.remove(self)
            awakened_tcell = ap.AgentList(self.model,1,ActivatedTCell)
            self.space.add_agents(awakened_tcell,positions = [pos])
            awakened_tcell.update_position()  
            self.model.agents.extend(awakened_tcell) 
           
class ActivatedTCell(TCell):
    
    def setup(self):
        self.idc = 4 
        self.phase = 0
        self.space = self.model.grid
        self.move_prob = self.p.move_prob_TCell
        self.duration = 0
        
        
        

    def fight(self):
        """
        delineates possible outcomes of immune-cancer interaction
        """
        self.neighbors = self.space.neighbors(self)
        detected_cancers = self.model.selection(self.neighbors, ProliferativeCancerCell)
        if not detected_cancers:
            return
        target = random.choice(detected_cancers)
        #killing probability with PDL1 == False is higher
        if target.PDL1 == False:
            if random.random() < self.p.pdl1_neg_death_prob:
                target.death()
            else:
                if random.random() < self.p.pdl1_switching_prob:
                    target.PDL1 = True
                    target.idc = 2  #changes identification to distinguish it from PDL1 negative in animation
                    return
                if random.random() < self.p.suppression_prob:
                    self.death()
                    return
        if target.PDL1 == True:
            if random.random() < self.p.pdl1_pos_death_prob:
                target.death()
                return
            else:
                if random.random() < self.p.suppression_prob:
                    self.death()            
                    return

class CancerModel(ap.Model):

    def O2_diffusion(self,u,entry_points,cell_positions):
        production_rate = self.p.o2_production_rate
        consumption_rate = self.p.o2_consumption_rate
        diffusion_rate =self.p.o2_diffusion_rate
        size = self.p.size
        dx = self.p.dx
        dt = self.p.time_step
        boundary_condition = self.p.o2_boundary_condition
        c, v = zip(*cell_positions)
        #Production
        u[entry_points[:, 0], entry_points[:, 1]] += production_rate
        #Consumption
        u[c,v] -= consumption_rate*u[c,v]
        # Compute Laplacian 
        lap = (np.roll(u, -1, axis=0) + np.roll(u, 1, axis=0) + np.roll(u, -1, axis=1) + np.roll(u, 1, axis=1) - 4*u)/dx**2 
        # Update the solution for the interior points
        un = u.copy()
        un[1:size-1, 1:size-1] = u[1:size-1, 1:size-1] + diffusion_rate*lap[1:size-1, 1:size-1]*dt
        #for each side set boundary condition
        un[:, 0] = boundary_condition
        un[:, -1] = boundary_condition
        un[0, :] = boundary_condition
        un[-1, :] = boundary_condition

        return un

    def IL2_diffusion(self,u,activated_locations):
    
        if not activated_locations.size > 0:
            return u

        production_rate = self.p.il2_production_rate
        degradation_rate = self.p.il2_degradation_rate
        size = self.p.size
        dx = self.p.dx
        diffusion_rate = self.p.il2_diffusion_rate
        dt = self.p.time_step    
        boundary_condition = self.p.il2_boundary_condition
        #Production
        u[activated_locations[:, 0], activated_locations[:, 1]] += production_rate
        #Degradation
        u -= degradation_rate*u
        # Compute Laplacian
        lap = (np.roll(u, -1, axis=0) + np.roll(u, 1, axis=0) + np.roll(u, -1, axis=1) + np.roll(u, 1, axis=1) - 4*u)/dx**2 
        # Update the solution for the interior points
        un = u.copy()
        un[1:size-1, 1:size-1] = u[1:size-1, 1:size-1] + diffusion_rate*lap[1:size-1, 1:size-1]*dt
        #for each side set boundary condition
        un[:, 0] = boundary_condition 
        un[:, -1] = boundary_condition
        un[0, :] = boundary_condition
        un[-1, :] = boundary_condition
        
        return un

    def selection(self,objects, class_type):
        """ 
        selection is used every time i want to select agents of a certain class from model.agents to perform 
        a certain operation on the new agentlist
        """
        l = filter(lambda i: isinstance(i,class_type),objects)
        return ap.AgentList(self,l)

    def recruitment(self):
        k_a = self.p.mutational_burden
        k_i = self.p.neoantigen_strength
        r = self.p.recruitment_rate
        index = self.t - self.p.t_delay
        if index < 0:
            index = 0
        N = self.dead_cancer[index]

        return k_a*N*r/(1/k_i+N)

    def entry_points(self):
        """
        entry_points are where blood vessels connect to tumor microenvironment
        this is were tcells and oxygen are added to the simulation 
        """
        x_coords, y_coords = np.random.randint(0, self.p.size-1, self.p.n_entry_points), np.random.randint(0, self.p.size-1,  self.p.n_entry_points)
        selected_cells = list(zip(x_coords, y_coords))
        return selected_cells
    
    def setup(self):
        
        #variables
        # We keep track of number of dead cancer cells (either killed or starved) 
        # because number of T-cells depends on how many cancer cells have died. 
        self.dead_cancer = [0]  
        #create grid
        self.grid = ap.Grid(self, (self.p.size,self.p.size), track_empty = True)
        #create attribute fields on the grid to keep track of oxygen and il2
        self.grid.add_field("oxy",float(self.p.o2_initial_condition))
        self.grid.add_field("il2",float(self.p.il2_initial_condition))
        #create cancer cell at center of the grid
        self.agents = ap.AgentList(self,1,ProliferativeCancerCell)
        self.grid.add_agents(self.agents,[(int(self.p.size/2),int(self.p.size/2))])
        self.agents.update_position()
        #define entry_points
        self.entry_points = self.entry_points()
        
    def step(self):
        """ Cells age is updated """
        #update oxygen level of each cell 
        self.agents.oxygen_update()   
        #increase phase of cell cycle for Tcells and proliferative cancer
        self.t_cells = self.model.selection(self.agents,TCell)
        self.t_cells.aging()
        self.proliferative_cancer = self.model.selection(self.agents,ProliferativeCancerCell)
        self.proliferative_cancer.aging()
        #increase time quiescent cells have been asleep
        self.quiescent_cancer = self.model.selection(self.agents,QuiescentCancerCell)
        self.quiescent_cancer.age += self.p.time_step
        """ Cancer cells transition state based on oxygen levels """
        #quiescent wakes up 
        subset = self.quiescent_cancer.select(self.quiescent_cancer.oxy > self.p.wake_up_value)
        subset.wake_up()
        #proliferative go to sleep
        subset = self.proliferative_cancer.select(self.proliferative_cancer.oxy < self.p.go_sleep_value)
        subset.go_sleep()
        """ Death by oxygen depravation """
        #tcells die
        subset = self.t_cells.select(self.t_cells.oxy < self.p.tcell_die_value)
        subset.death()
        #quiescent cancer die, death count is updated
        self.quiescent_cancer = self.model.selection(self.agents,QuiescentCancerCell)
        subset = self.quiescent_cancer.select(self.quiescent_cancer.age > self.p.quiescent_duration)
        subset.death()
        """ Replication """
        #replication proliferative 
        self.proliferative_cancer = self.model.selection(self.agents,ProliferativeCancerCell) 
        subset = self.proliferative_cancer.select(self.proliferative_cancer.phase > 1)
        subset.replication()
        #replication tcells
        self.t_cells = self.model.selection(self.agents,TCell)
        subset = self.t_cells.select(self.t_cells.phase > 1)
        subset.replication()
        """ Movement """
        self.agents.move()
        """ Attack """
        subset = self.model.selection(self.t_cells, ActivatedTCell)
        subset.fight() 
        """ Activation """
        subset = self.model.selection(self.t_cells, EffectorTCell)
        subset.activation()
        """Tcell recruitment"""
        number = self.recruitment()  
        #following code assigns tcells to grid. Positions are given by entry point
        #which is iterated over until all new tcells are inserted
        while number > 0:
            random.shuffle(self.entry_points)    
            for i in self.entry_points:
                self.t_cell = ap.AgentList(self,1,EffectorTCell)
                distance = 1
                while True:
                    available = self.t_cell.empty_cells(i,distance)
                    if available[0]:
                        self.agents.extend(self.t_cell)
                        self.grid.add_agents(self.t_cell,[random.choice(available[0])])
                        self.t_cell.update_position()
                        break
                    distance += 1 # if no empty_cell available, try with larger neighborhood
                number -= 1
                if number < 0:
                    break

        """ Diffusive step IL-2 """ 
        #select recently activated TCell whose positions will be passed to PDE solver
        subset = self.model.selection(self.agents, ActivatedTCell)
        producers = subset.select(subset.duration < self.p.release_duration)
        producers.duration += self.p.time_step
        pos = [list(tup) for tup in producers.pos]
        self.grid.il2 = self.IL2_diffusion(self.grid.il2,np.array(pos))
        """ Diffusive step O2 """
        #all cells positions are passed to PDE solver
        cell_positions = list(self.grid.positions.values())
        self.grid.oxy = self.O2_diffusion(self.grid.oxy,np.array(self.entry_points),cell_positions)
        self.dead_cancer.append(0)


p = {'steps':200,'time_step':1,'size':101,  #steps è il numero di iterazioni. time_step è l'unità fondamentale di tempo, che corrisponde ad un'iterazione
        'n_entry_points':150,  #numero di punti di entrata di ossigeno e cellule T
        'C_phase':3, 'Tmin':3, #Tmin è direttamente proporzionale al tempo di replicazione
         
        'wake_up_value': 5,'go_sleep_value':3,'tcell_die_value':6, #Valori di concentrazione di ossigeno per cambiamenti di stato cellule
        'quiescent_duration':5,  #Durata dello stato quiescente in termini di time_steps
        'move_prob_TCell':0.95,'move_prob_Cancer':0.01, #Probabilità di movimento

        'pdl1_pos_death_prob':0.2,'pdl1_neg_death_prob':0.4, #probabilità legate all'interazione tumore-Tcell
        'pdl1_switching_prob':0.8,'suppression_prob':0.1,

        'mutational_burden':15,'neoantigen_strength':0.01,'recruitment_rate':1, #fattori che influenzano la velocità di recruitment di cellule T
        'il2_threshold':20,   #concentrazione IL-2 necessaria per replicazione cellule T

        'dx':1,   #incremento spaziale 
        't_delay':1,'release_duration':4,
        'il2_degradation_rate':0.2,'o2_consumption_rate':0.1,     #parametri legati alle PDE
        'il2_production_rate':200,'o2_production_rate':200,
        'il2_diffusion_rate':0.4,'o2_diffusion_rate':0.25,
        'il2_initial_condition':0,'o2_initial_condition':5,
        'il2_boundary_condition':0,'o2_boundary_condition':20}
    

def animation_plot(m, ax):
    ax1, ax2, ax3 = ax
    ax1.set_title(f"Cancer Model \n Time-step: {m.t} ")
    ax2.set_title("Oxygen")
    ax3.set_title("IL-2")

    cancer_grid = m.grid.attr_grid('idc')
    ap.gridplot(cancer_grid, cmap='Accent', ax=ax1)

    oxy_grid = m.grid.oxy
    ap.gridplot(oxy_grid, cmap='viridis', ax=ax2)

    il2_grid = m.grid.il2
    ap.gridplot(il2_grid, cmap='viridis', ax=ax3)
    
fig, ax = plt.subplots(1, 3,figsize=(15,5)) 
v = CancerModel(p)
animation = ap.animate(v, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())